# 데이터 전처리

## #01. 패키지 참조

In [1]:
import numpy as np
from pandas import read_csv, concat, merge, DataFrame
from matplotlib import pyplot as plt
import seaborn as sb

## #02. 데이터 구축

### 1. 데이터 파일 열기

In [3]:
df2018 = read_csv('서울시 우리마을가게 상권분석서비스(상권-추정매출)_2018.csv', encoding='euc-kr')
df2019 = read_csv('서울시 우리마을가게 상권분석서비스(상권-추정매출)_2019.csv', encoding='euc-kr')
df2020 = read_csv('서울시 우리마을가게 상권분석서비스(상권-추정매출)_2020.csv', encoding='euc-kr')
df2021 = read_csv('서울시 우리마을가게 상권분석서비스(상권-추정매출)_2021.csv', encoding='euc-kr')
df상권영역 = read_csv('서울시 우리마을가게 상권분석서비스(상권영역).csv', encoding='euc-kr')
df건축물 = read_csv('서울특별시 건축물대장 법정동 코드정보.csv', encoding='euc-kr')

### 2. 상권영역에 구 이름 병합

#### 상권영역 데이터

서울시에만 해당하는 데이터

In [4]:
df상권영역.head()

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보
0,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN
1,201810,R,전통시장,1001454,봉천제일종합시장,195147,442413,11620,11620595,NaN
2,201810,R,전통시장,1001474,도곡시장,204551,444227,11680,11680650,NaN
3,201810,R,전통시장,1001475,강남개포시장,206065,443310,11680,11680670,NaN
4,201810,R,전통시장,1001412,화곡본동시장,186203,449328,11500,11500590,NaN


#### 법정동 코드정보

전국에 대한 데이터

In [5]:
df건축물.head()

,시군구코드,법정동코드,행정동코드,시도명,시군구명,법정동명,행정동명,적용시작일,적용만료일
0,42750,31224,0,강원도,영월군,산솔면 화원리,NaN,20211102,99991231
1,42750,31226,0,강원도,영월군,산솔면 석항리,NaN,20211102,99991231
2,42750,31221,0,강원도,영월군,산솔면 녹전리,NaN,20211102,99991231
3,42750,31223,312,강원도,영월군,산솔면 이목리,산솔면,20211102,99991231
4,42750,31225,312,강원도,영월군,산솔면 연상리,산솔면,20211102,99991231


#### 법정동 코드 정보를 상권영역에 결합

In [6]:
df상권영역2 = merge(df상권영역, df건축물.filter(['시군구명', '시군구코드']), left_on='시군구_코드', right_on='시군구코드', how='inner')
df상권영역2

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보,시군구명,시군구코드
0,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620
1,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620
2,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620
3,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620
4,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620
...,...,...,...,...,...,...,...,...,...,...,...,...
145707,201810,A,골목상권,1000381,동일로242길,205175,463916,11350,11350630,NaN,노원구,11350
145708,201810,A,골목상권,1000381,동일로242길,205175,463916,11350,11350630,NaN,노원구,11350
145709,201810,A,골목상권,1000381,동일로242길,205175,463916,11350,11350630,NaN,노원구,11350
145710,201810,A,골목상권,1000381,동일로242길,205175,463916,11350,11350630,NaN,노원구,11350


### 3. `구`에 대한 정보 확인

#### 얼마나 많은 `구`가 있는가?

In [7]:
df상권영역2['시군구명'].value_counts()

시군구명
종로구     14553
강남구     13915
영등포구    12284
성북구     11868
중구      11825
마포구      7739
성동구      7728
관악구      7568
동대문구     7268
용산구      5858
강서구      5313
구로구      4897
서초구      4760
서대문구     3936
강동구      3480
동작구      3250
은평구      3111
송파구      2925
양천구      2646
도봉구      2508
중랑구      2304
광진구      2040
강북구      2028
노원구      1170
금천구       738
Name: count, dtype: int64

#### 파생변수 생성

`행정구`는 값의 종류가 너무 많아서 더미변수화 하기가 어려울 것으로 보고 도심과 비도심으로 구분하는 파생변수를 생성

> 출처: 서울2030플렌

In [8]:
도심 = ['종로구', '중구', '영등포구', '서초구', '강남구'] 

df상권영역2['도심'] = np.isin(df상권영역2['시군구명'], 도심)

df상권영역2.head()

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보,시군구명,시군구코드,도심
0,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620,False
1,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620,False
2,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620,False
3,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620,False
4,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620,False


In [9]:
df상권영역2.to_excel('df상권영역2.xlsx', index=False)